<a href="https://colab.research.google.com/github/sou-venir/Dream-nori/blob/main/%EC%BD%94%EB%9E%A9%EC%9D%84%EC%9C%84%ED%95%9C%EB%93%9C%EB%A6%BC%EB%86%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# [1] 필수 라이브러리 설치 (코랩 환경 전용)
!pip install flask-socketio flask-ngrok pyngrok openai python-socketio eventlet
!fuser -k 5000/tcp # 기존에 사용 중인 포트가 있다면 강제 종료

import os
import json
from flask import Flask, render_template_string
from flask_socketio import SocketIO, emit
from pyngrok import ngrok
from google.colab import userdata
import openai
from google.colab import drive

# [2] 구글 드라이브 마운트 (채팅 로그 및 설정값 영구 저장)
drive.mount('/content/drive')

# 저장 경로 설정: 내 드라이브의 'ChatData' 폴더
SAVE_PATH = '/content/drive/MyDrive/ChatData'
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

# [3] 보안 및 AI 설정
# 코랩 왼쪽 '열쇠' 아이콘 메뉴에서 NGROK_AUTH_TOKEN과 OPENAI_API_KEY를 추가해야 합니다! 발급받은 키를 설정해 주세요
try:
    NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    ngrok.set_auth_token(NGROK_TOKEN)
except Exception as e:
    print(f"❌ 시크릿 설정 확인 필요: {e}")

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")

DATA_FILE = os.path.join(SAVE_PATH, "save_data.json")

# [저장 로직] 상태가 변할 때마다 구글 드라이브에 JSON 파일로 기록
def save_data():
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(state, f, ensure_ascii=False, indent=4)

# [불러오기 로직] 서버 시작 시 기존 저장된 데이터를 가져옴
def load_data():
    if os.path.exists(DATA_FILE):
        try:
            with open(DATA_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except:
            return None
    return None

# [4] 서버 상태 관리 (기본값 설정)
initial_state = {
    "session_title": "드림놀이",
    "theme": {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"},
    "accent_color": "#e91e63",
    "admin_password": "3896", # 마스터 설정 진입용 기본 비밀번호!!
    "is_locked": False,
    "inputs": {"user1": None, "user2": None},
    "profiles": {
        "user1": {"name": "Player 1", "bio": "", "canon": ""},
        "user2": {"name": "Player 2", "bio": "", "canon": ""}
    },
    "history": [],
    "ai_history": [],
    "summary": "기록된 줄거리가 없습니다.",
    "prologue": "이곳에 프롤로그를 입력하세요.",
    "sys_prompt": "당신은 숙련된 TRPG 마스터입니다.",
    "lorebook": []
}

saved_state = load_data()
state = saved_state if saved_state else initial_state

# [5] AI 테마 분석 로직
def analyze_theme_color(title, sys_prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "system",
                "content": "너는 웹 디자인 전문가야. 세션 설정에 어울리는 테마 색상을 골라줘. 모든 UI 텍스트는 검은색이야. JSON 형식으로 답변해: {\"bg\": \"배경색\", \"panel\": \"패널색\", \"accent\": \"강조색\"}"
            }, {
                "role": "user",
                "content": f"제목: {title}\n설정: {sys_prompt}"
            }],
            response_format={ "type": "json_object" }
        )
        palette = json.loads(response.choices[0].message.content)
        default = {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"}
        default.update(palette)
        return default
    except:
        return {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"}

# [6] 프론트엔드 HTML 템플릿
HTML_TEMPLATE = """<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>드림놀이</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.1/socket.io.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
    <style>
        :root {
            --bg: {{ theme.bg }};
            --panel: {{ theme.panel }};
            --accent: {{ theme.accent }};
        }
        html, body { height: 100%; margin: 0; overflow: hidden; font-family: sans-serif; }
        body { display: flex; background: var(--bg); color: #000; }
        #main { flex: 1; display: flex; flex-direction: column; height: 100vh; }
        #chat-window { flex: 1; overflow-y: auto; padding: 20px 10%; display: flex; flex-direction: column; gap: 15px; }
        #sidebar { width: 320px; background: var(--panel); padding: 20px; overflow-y: auto; display: flex; flex-direction: column; gap: 12px; }
        .bubble { padding: 15px; border-radius: 12px; max-width: 85%; line-height: 1.6; background: rgba(0,0,0,0.03); }
        .user-bubble { align-self: flex-end; border-right: 5px solid var(--accent); background: var(--bg); }
        .center-ai { align-self: center; width: 100%; max-width: 800px; border-left: 5px solid var(--accent); background: var(--panel); }
        textarea, input { width: 100%; box-sizing: border-box; padding: 10px; border-radius: 8px; border: 1px solid #ccc; }
        button { cursor: pointer; padding: 10px; border: none; border-radius: 8px; background: var(--accent); color: #fff; font-weight: bold; }
        #admin-modal { display: none; position: fixed; top: 0; left: 0; width: 100%; height: 100%; background: rgba(0,0,0,0.5); justify-content: center; align-items: center; z-index: 9999; }
        .modal-content { background: #fff; padding: 25px; border-radius: 15px; width: 90%; max-width: 600px; max-height: 80vh; overflow-y: auto; }
        .tab-content { display: none; flex-direction: column; gap: 10px; }
        .tab-content.active { display: flex; }
    </style>
</head>
<body>
    <div id="main">
        <div id="chat-window"><div id="chat-content"></div></div>
        <div style="padding:20px;">
            <div id="status" style="font-size:12px; color:var(--accent); font-weight:bold; margin-bottom:5px;">대기 중</div>
            <div style="display:flex; gap:10px;">
                <textarea id="msg-input" placeholder="메시지 입력... (잠금 버튼 클릭 후 가능)"></textarea>
                <button onclick="send()">전송</button>
            </div>
        </div>
    </div>
    <div id="sidebar">
        <h3>🎭 설정</h3>
        <select id="user-role" onchange="refreshUI()">
            <option value="user1">Player 1</option>
            <option value="user2">Player 2</option>
        </select>
        <input type="text" id="p-name" placeholder="이름">
        <textarea id="p-bio" placeholder="캐릭터 설정"></textarea>
        <textarea id="p-canon" placeholder="드림캐 설정"></textarea>
        <button onclick="saveProfile()">설정 저장</button>
        <button onclick="confirmLock()">🔒 설정 완료 및 잠금</button>
        <button onclick="requestAdmin()" style="background:#888; margin-top:auto;">⚙️ 마스터 설정</button>
    </div>

    <div id="admin-modal">
        <div class="modal-content">
            <div style="display:flex; gap:5px; margin-bottom:15px;">
                <button onclick="openTab('t-base')">시스템</button>
                <button onclick="openTab('t-lore')">키워드북</button>
                <button onclick="closeModal()" style="margin-left:auto; background:#888;">닫기</button>
            </div>
            <div id="t-base" class="tab-content active">
                <input type="text" id="m-title" placeholder="세션 제목">
                <textarea id="m-sys" style="height:150px;" placeholder="시스템 프롬프트"></textarea>
                <textarea id="m-pro" style="height:100px;" placeholder="프롤로그"></textarea>
                <textarea id="m-sum" placeholder="줄거리 요약"></textarea>
                <button onclick="saveMaster()">저장 및 테마 업데이트</button>
                <button onclick="sessionReset()" style="background:red; margin-top:10px;">⚠️ 전체 초기화</button>
            </div>
            <div id="t-lore" class="tab-content">
                <input type="text" id="kw-t" placeholder="키워드">
                <input type="text" id="kw-tr" placeholder="트리거">
                <textarea id="kw-c" placeholder="내용"></textarea>
                <button onclick="addLore()">키워드 추가</button>
                <div id="lore-list"></div>
            </div>
        </div>
    </div>

    <script>
        const socket = io();
        let gState = null;

        socket.on('status_update', d => { document.getElementById('status').innerText = d.msg; });
        socket.on('initial_state', data => {
            gState = data;
            if(data.theme) {
                const r = document.documentElement.style;
                r.setProperty('--bg', data.theme.bg);
                r.setProperty('--panel', data.theme.panel);
                r.setProperty('--accent', data.theme.accent);
            }
            refreshUI();
        });

        function refreshUI() {
            if(!gState) return;
            renderChat();
            renderLore();
            const role = document.getElementById('user-role').value;
            const p = gState.profiles[role];
            if(document.activeElement.tagName !== 'TEXTAREA' && document.activeElement.tagName !== 'INPUT') {
                document.getElementById('p-name').value = p.name || "";
                document.getElementById('p-bio').value = p.bio || "";
                document.getElementById('p-canon').value = p.canon || "";
                document.getElementById('m-title').value = gState.session_title || "";
                document.getElementById('m-sys').value = gState.sys_prompt || "";
                document.getElementById('m-pro').value = gState.prologue || "";
                document.getElementById('m-sum').value = gState.summary || "";
            }
        }

        function renderChat() {
            let h = `<div style="text-align:center; padding:10px; font-weight:bold; color:var(--accent);">${gState.session_title}</div>`;
            h += `<div class="bubble center-ai"><b>[PROLOGUE]</b><br>${marked.parse(gState.prologue || "")}</div>`;
            gState.ai_history.forEach(msg => {
                const name = gState.profiles[document.getElementById('user-role').value].name;
                const isUser = name && msg.includes(`**${name}**:`);
                h += `<div class="bubble ${isUser ? 'user-bubble' : 'center-ai'}">${marked.parse(msg)}</div>`;
            });
            document.getElementById('chat-content').innerHTML = h;
            const win = document.getElementById('chat-window'); win.scrollTop = win.scrollHeight;
        }

        function send() {
            const input = document.getElementById('msg-input');
            if(!input.value.trim() || !gState.is_locked) return;
            socket.emit('client_message', { uid: document.getElementById('user-role').value, text: input.value });
            input.value = '';
        }

        function confirmLock() { if(confirm("설정을 완료하시겠습니까?")) socket.emit('lock_settings'); }
        function requestAdmin() { const pw = prompt("관리자 암호:"); if(pw) socket.emit('check_admin', { password: pw }); }
        socket.on('admin_auth_res', d => { if(d.success) document.getElementById('admin-modal').style.display = 'flex'; else alert("비밀번호 불일치"); });
        function saveProfile() { socket.emit('update_profile', { uid: document.getElementById('user-role').value, name: document.getElementById('p-name').value, bio: document.getElementById('p-bio').value, canon: document.getElementById('p-canon').value }); alert("저장됨"); }
        function saveMaster() { socket.emit('save_master_base', { title: document.getElementById('m-title').value, sys: document.getElementById('m-sys').value, pro: document.getElementById('m-pro').value, sum: document.getElementById('m-sum').value }); closeModal(); }
        function addLore() { socket.emit('add_lore', { title: document.getElementById('kw-t').value, triggers: document.getElementById('kw-tr').value, content: document.getElementById('kw-c').value }); }
        function renderLore() { document.getElementById('lore-list').innerHTML = gState.lorebook.map((l, i) => `<div style="padding:5px; border-bottom:1px solid #eee;"><b>${l.title}</b> <button onclick="socket.emit('del_lore', {index:${i}})" style="padding:2px 5px; font-size:10px; background:red;">X</button></div>`).join(''); }
        function openTab(id) { document.querySelectorAll('.tab-content').forEach(c => c.classList.remove('active')); document.getElementById(id).classList.add('active'); }
        function closeModal() { document.getElementById('admin-modal').style.display = 'none'; }
        function sessionReset() { if(confirm("초기화하시겠습니까?")) { socket.emit('reset_session', { password: prompt("암호 확인:") }); } }
        socket.emit('request_data');
    </script>
</body>
</html>
"""

# [7] 서버 측 소켓 통신 핸들러
@socketio.on('request_data')
def handle_request():
    emit('initial_state', state)

@socketio.on('lock_settings')
def on_lock_settings():
    p1 = state["profiles"].get("user1", {})
    p2 = state["profiles"].get("user2", {})
    if not p1.get("name") or not p2.get("name"):
        emit('status_update', {'msg': '❌ 양쪽 이름이 모두 있어야 시작 가능합니다.'})
        return
    state["is_locked"] = True
    save_data()
    emit('initial_state', state, broadcast=True)

@socketio.on('client_message')
def on_client_message(data):
    user_text = data.get('text', '').strip()
    uid = data.get('uid')

    # 키워드 필터링
    active_context = [f"[{l['title']}]: {l['content']}" for l in state.get('lorebook', []) if any(t.strip() in user_text for t in l.get('triggers', '').split(','))]

    messages = [{"role": "system", "content": f"{state['sys_prompt']}\n\n[줄거리]: {state['summary']}\n[참고]: {' '.join(active_context[:3])}"}]

    # 최근 기록 15개 추가
    for h in state['ai_history'][-15:]:
        messages.append({"role": "assistant" if h.startswith("**AI**") else "user", "content": h})

    current_name = state['profiles'].get(uid, {}).get('name', 'User')
    messages.append({"role": "user", "content": f"{current_name}: {user_text}"})

    try:
        emit('status_update', {'msg': '🤔 AI가 서술 중...'})
        response = client.chat.completions.create(model="gpt-4o", messages=messages)
        ai_response = response.choices[0].message.content

        state["ai_history"].append(f"**{current_name}**: {user_text}")
        state["ai_history"].append(f"**AI**: {ai_response}")

        save_data()
        emit('initial_state', state, broadcast=True)
        emit('status_update', {'msg': '✅ 응답 완료'})
    except Exception as e:
        emit('status_update', {'msg': f'❌ 에러: {str(e)}'})

@socketio.on('save_master_base')
def on_save_master(data):
    state.update({"session_title": data['title'], "sys_prompt": data['sys'], "prologue": data['pro'], "summary": data['sum']})
    state['theme'] = analyze_theme_color(state['session_title'], state['sys_prompt'])
    save_data()
    emit('initial_state', state, broadcast=True)

@socketio.on('update_profile')
def on_profile(data):
    uid = data.get('uid')
    state["profiles"][uid].update({"name": data['name'], "bio": data['bio'], "canon": data['canon']})
    save_data()
    emit('initial_state', state, broadcast=True)

@socketio.on('add_lore')
def on_add_lore(data):
    state.setdefault("lorebook", []).append(data)
    save_data(); emit('initial_state', state, broadcast=True)

@socketio.on('del_lore')
def on_del_lore(data):
    state["lorebook"].pop(data['index'])
    save_data(); emit('initial_state', state, broadcast=True)

@socketio.on('check_admin')
def check_admin(data):
    emit('admin_auth_res', {'success': str(data.get('password')) == str(state.get('admin_password'))})

@socketio.on('reset_session')
def on_reset_session(data):
    if str(data.get('password')) == str(state.get('admin_password')):
        state.update({"ai_history": [], "lorebook": [], "is_locked": False})
        save_data(); emit('initial_state', state, broadcast=True)

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE, theme=state['theme'], session_title=state['session_title'])

# [8] 서버 실행
if __name__ == '__main__':
    ngrok.kill()
    public_url = ngrok.connect(5000).public_url
    print(f"\n🚀 접속 주소: {public_url}\n")
    socketio.run(app, port=5000, allow_unsafe_werkzeug=True)